In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt

import geopandas as gpd
import pyproj as proj


In [5]:
def quantile_exc(ser, q):
    ser_sorted = ser.squeeze().sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - 
                                     ser_sorted.iat[rank_l]) * (rank - rank_l)

In [7]:
metadata = pd.read_csv('turkeyhill_locationmetadata.txt',delimiter='\t',header=59)
metadata = metadata.drop(0)

ddata = pd.read_csv('discharge_turkeycreek.txt',delimiter='\t',header=29,names=
                   ['agency_code',
                   'site_no',
                   'date',
                   'discharge_cfs',
                   'approval'],parse_dates=[2])


In [9]:
#import requested watershed shapefile delineated from StreamStats
watershed = gpd.read_file('globalwatershed.shp')
statewide_lambert = proj.CRS.from_proj4('+proj=lcc +lat_0=0 +lon_0=-83.5 +lat_1=31.4166666666667 +lat_2=34.2833333333333 +x_0=0 +y_0=0 +datum=NAD83 +units=us-ft +no_defs +type=crs')
watershed = watershed.to_crs(statewide_lambert)

upstream_watershed = gpd.read_file('upstream_globalwatershed.shp')
upstream_watershed = upstream_watershed.to_crs(statewide_lambert)

basin_area = metadata.contrib_drain_area_va
withdrawal_area = watershed.area * 3.587E-8 #convert from sq ft to sq miles
upstream_area = upstream_watershed.area* 3.587E-8 #sq mi
diff_in_area = withdrawal_area-upstream_area

In [10]:
#upstream demands of irrigated area, from external document
irrigated_area = 215
max_reservoir_volume = 28.224 #cubic feet
pumprate = 1400 * 60 * 24 * 1.547/1e6 #gpm, converted to cfs (min/hr * hr/day * 1.547 cfs*day^-1/1e6 gallons*day^-1/)
annual_demand = 15 * 250 /12 #af 

upstream_demands = pd.Series( #standardized inch/yr*acre demands (total15 in/yr), multiplied by acres irrigated area, converted to cfs
[0.227,
0.339,
0.603,
0.795,
1.82,
2.269,
2.273,
2.717,
1.564,
1.349,
0.685,
0.36]).multiply(irrigated_area)
month_range = pd.date_range(start='01-01-2021',end='01-01-2022', freq='M')
inches_in_feet = 12
ftsq_in_acre = 43560
seconds_in_day = 24*60*60
#convert to cfs
upstream_demands = upstream_demands.multiply(ftsq_in_acre).divide(inches_in_feet).divide(month_range.days_in_month).divide(seconds_in_day)

In [11]:
#Evaporative Loss
reservoir_surface_area = 7#acres
evaporation_rate = pd.read_csv('acfbasin_evapnetrate.csv',header=6,parse_dates=True,usecols=[1,2],names=['date','rate'])
#multiply by reservoir area, convert to cfs
evaporative_loss = pd.Series(data = 
                             evaporation_rate.rate.values*reservoir_surface_area*1/inches_in_feet*ftsq_in_acre*1/seconds_in_day,
                             index = evaporation_rate.date.astype(np.datetime64))

dummy = pd.Series(np.zeros(ddata.shape[0]),ddata.date)
evaporative_loss = pd.Series.combine(evaporative_loss,dummy, max,fill_value=0)

In [12]:
#scaled by upstream drainage area of withdrawal site, divided by river catchment area
ddata['scaled_by_area']=(ddata.discharge_cfs*(np.float(upstream_area)/np.float(basin_area)))
#scaled by area difference btw upstream and downstream drainage areas of withdrawal site, divided by river catchment area
ddata['scaled_by_diff']=(ddata.discharge_cfs*(np.float(diff_in_area)/np.float(basin_area)))

In [13]:
withdrawal = np.zeros([len(ddata.scaled_by_diff),1])
for i in np.arange(0,withdrawal.shape[0]):
    month = ddata.date[i].month
    withdrawal[i] = max(ddata.scaled_by_area.values[i]-upstream_demands[month-1],0)
    #timeseries of discharge = max(0,withdrawal)+scaled discharge
inflow_data = np.add(ddata.scaled_by_diff.values,np.transpose(withdrawal))

In [14]:
inflow_ts = pd.Series(data = inflow_data.squeeze(),index = ddata.date) #data used for

In [15]:
inflow_ts=inflow_ts[:np.argmin(ddata.date.le(dt.datetime(2022,6,30)))]
inflow_ts

date
1958-06-20    0.140436
1958-06-21    0.166767
1958-06-22    0.105327
1958-06-23    0.087772
1958-06-24    0.083384
                ...   
2022-06-26    0.019090
2022-06-27    0.018169
2022-06-28    0.017818
2022-06-29    0.035592
2022-06-30    0.027166
Length: 23387, dtype: float64

In [16]:
# Moving 7 day Avg - translated from Excel 
nmonth = np.zeros([12,1],dtype=int)
days7Ave = np.zeros([len(inflow_ts),1])

for j in np.arange(0,11):
    nmonth[j] = 0
    
    j=0
    for i in np.arange(0, len(inflow_ts)-1):
        j = j+1
        if j < 6:
            vsum=0
            for k in np.arange(j,1,-1):
                vsum = vsum + inflow_ts.values[k]
            
            days7Ave[j] = vsum/j
        else:
            vsum = 0
            for k in np.arange(j, j-6, -1):
                vsum = vsum + inflow_ts.values[k-1]
            days7Ave[j-1] = vsum/7
            
    

# Monthly minimum - translated from Excel
mm7dayAve = np.zeros([12, len(np.unique(inflow_ts.index.year)), 2])
tmm7dave = days7Ave[0]

for k in np.arange(0, len(inflow_ts)-1):
    if inflow_ts.index[k].month == inflow_ts.index[k-1].month:
        if days7Ave[k] < tmm7dave:
            tmm7dave = days7Ave[k]
    else:
        j = inflow_ts.index[k].month
        nmonth[j-1] = nmonth[j-1]+1
        mm7dayAve[j-1,nmonth[j-1],0] = tmm7dave
        mm7dayAve[j-1,nmonth[j-1],1] = inflow_ts.index[k].toordinal()
        tmm7dave  = days7Ave[k-1]
    #end-of-record
    if k==len(inflow_ts)-1:
        j = inflow_ts.index[k].month
        nmonth[j-1] = nmonth[j-1]
        mm7dayAve[j-1,nmonth[j-1],0] = tmm7dave
        mm7dayAve[j-1,nmonth[j-1],1] = inflow_ts.index[k].toordinal()
        

    
    
    

In [17]:
nmonth

array([[64],
       [64],
       [64],
       [64],
       [64],
       [64],
       [64],
       [64],
       [64],
       [64],
       [64],
       [64]])

In [18]:
M7Q10=np.zeros([12,1])
for m in np.arange(0,12):
    mm7dayAve2 = np.zeros([int(nmonth[m]),1])
      
    for j in np.arange(0, nmonth[m]):
        mm7dayAve2[j] = mm7dayAve[m, j, 0]
        
    
    M7Q10[m] = quantile_exc(pd.Series(mm7dayAve2[1:].squeeze()),0.1)

In [19]:
M7Q10

array([[0.01551188],
       [0.02371427],
       [0.03799579],
       [0.05852939],
       [0.02793668],
       [0.01445987],
       [0.01122232],
       [0.00957596],
       [0.00870325],
       [0.0087835 ],
       [0.00837724],
       [0.01189942]])

In [20]:
pd.Timestamp.fromordinal(mm7dayAve[0,1,1].astype(int))

Timestamp('1959-01-01 00:00:00')

In [21]:
#Pythonic
weekly_inflow_ts = inflow_ts.rolling(7,center=True).mean()#weekly resample by median

monthly_min_inflow_ts = weekly_inflow_ts.resample('1M').min() #monthly min of 7-day averages
annual_min_inflow_ts = weekly_inflow_ts.resample('1Y').min() #annual min of 7-day averages

In [22]:
monthly_min_inflow_ts

date
1958-06-30    0.048212
1958-07-31    0.036426
1958-08-31    0.033479
1958-09-30    0.024576
1958-10-31    0.024012
                ...   
2022-02-28    0.244062
2022-03-31    0.158367
2022-04-30    0.093959
2022-05-31    0.040808
2022-06-30    0.019993
Freq: M, Length: 769, dtype: float64

In [23]:
#10th percentile by month
M7Q10 = monthly_min_inflow_ts.groupby(monthly_min_inflow_ts.index.month).aggregate(quantile_exc,q=0.1) 
#number of months included in calculation
nmonths = monthly_min_inflow_ts.groupby(monthly_min_inflow_ts.index.month).aggregate(np.count_nonzero) 

#10th percentile by year
A7Q10 = quantile_exc(annual_min_inflow_ts,0.1)

In [24]:
M7Q10

date
1     0.032084
2     0.054619
3     0.068101
4     0.030971
5     0.016865
6     0.013201
7     0.011244
8     0.010157
9     0.010461
10    0.010338
11    0.015407
12    0.018373
dtype: float64

In [25]:
A7Q10.round(3)

0.009

In [26]:
M7Q10_flow = pd.Series(np.zeros(inflow_ts.shape),inflow_ts.index) #MIF, flow timeseries where each day's flow = monthly M7Q10 
for month in np.arange(1,12):
    M7Q10_flow[M7Q10_flow.index.month == month] = M7Q10[month]

In [27]:
#when inflow < M7Q10, then in-stream outflow = inflow, otherwise outflow = M7Q10
instream_flow = M7Q10_flow 
instream_flow[inflow_ts<=M7Q10_flow] = inflow_ts
instream_flow

date
1958-06-20    0.013201
1958-06-21    0.013201
1958-06-22    0.013201
1958-06-23    0.013201
1958-06-24    0.013201
                ...   
2022-06-26    0.013201
2022-06-27    0.013201
2022-06-28    0.013201
2022-06-29    0.013201
2022-06-30    0.013201
Length: 23387, dtype: float64

In [28]:
available_flow = inflow_ts-instream_flow
available_flow

date
1958-06-20    0.127235
1958-06-21    0.153566
1958-06-22    0.092126
1958-06-23    0.074571
1958-06-24    0.070183
                ...   
2022-06-26    0.005890
2022-06-27    0.004968
2022-06-28    0.004617
2022-06-29    0.022391
2022-06-30    0.013965
Length: 23387, dtype: float64

In [29]:
pump_demand = pd.Series(data = np.zeros(inflow_ts.shape[0]), index=inflow_ts.index)

pump_demand[np.bitwise_or(pump_demand.index.month>=4 , pump_demand.index.month<=9)] = pumprate
pump_demand

date
1958-06-20    3.118752
1958-06-21    3.118752
1958-06-22    3.118752
1958-06-23    3.118752
1958-06-24    3.118752
                ...   
2022-06-26    3.118752
2022-06-27    3.118752
2022-06-28    3.118752
2022-06-29    3.118752
2022-06-30    3.118752
Length: 23387, dtype: float64

In [30]:
## Time Varying Model

In [31]:
available_water_vol = np.zeros([inflow_ts.shape[0]+1,1]) #s1
endofday_stored_water = np.zeros([inflow_ts.shape[0]+1,1]) #storage
pumped_water = np.zeros([inflow_ts.shape[0]+1,1]) #pumped
left_in_stream = np.zeros([inflow_ts.shape[0]+1,1]) #remaining, what stream water is left after pumping 
pump_gap  = np.zeros([inflow_ts.shape[0]+1,1])  #gap between pump demand and what is pumped

endofday_stored_water[0] = max_reservoir_volume
#available_water_vol[0] = inflow_ts[0]+max_reservoir_volume - instream_flow[0] - evaporative_loss[0]
#pumped_water[0] = max(min(available_water_vol[0],pump_demand[0]),0)
#left_in_stream[0] = instream_flow[0]  + max(available_water_vol[0]-pumped_water[0]-max_reservoir_volume,0)
#pump_gap[0] = pump_demand[0]-pumped_water[0]

In [32]:
for t in np.arange(1, inflow_ts.shape[0]-1):
    available_water_vol[t] = inflow_ts.values[t-1]+endofday_stored_water[t-1] - instream_flow.values[t-1] - evaporative_loss.values[t-1]
    pumped_water[t] = max(min(available_water_vol[t],pump_demand.values[t-1]),0)
    left_in_stream[t] = instream_flow.values[t-1]  + max(available_water_vol[t]-pumped_water[t]-max_reservoir_volume,0)
    pump_gap[t] = pump_demand[t]-pumped_water[t]
    endofday_stored_water[t] = max(min(available_water_vol[t]-pumped_water[t],max_reservoir_volume),0)

available_water_vol = available_water_vol[1:]
endofday_stored_water = endofday_stored_water[1:]
pumped_water = pumped_water[1:]
left_in_stream = left_in_stream[1:]
pump_gap  = pump_gap[1:]

In [33]:
pumped_water_ts = pd.Series(data=pumped_water.squeeze(), index = inflow_ts.index)

In [34]:
(pumped_water_ts*3.06888785).groupby(pumped_water_ts.index.year).sum()

date
1958    118.675370
1959    569.841500
1960    556.917662
1961    317.958336
1962    556.800379
           ...    
2018    415.184716
2019    458.232247
2020    794.040696
2021    614.637972
2022    266.091688
Length: 65, dtype: float64